In [42]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# // %%javascript
# // IPython.OutputArea.prototype._should_scroll = function(lines) {
# //     return false;
# // }

# Occasions Annotations

## Bin of 6+ `guest_count`
Let's look at a few tables:
- `hockey_3_text_processed.csv`
- `silvester_3_text_processed.csv`
- `valentine_3_text_processed.csv`

to get an idea of the different occasions.

###### Our initial set of **occasions** for the category 6+ bin is: 
1. `Drinking` - purpose to drink
    - `Drinking`
    - `Party`
    
    
2. `Meal` - purpose is to eat
    - `Dinner`
    - `Lunch`
    
    
3. `Social gathring` 
    - `after work` 
    
    
4. `Family Occasion` 

    


### Features:
1. time of day
2. food to drinks ratio
3. kid meal
4. meal step



----------

In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 50)

In [13]:
hockey_path = '../data/hockey_3_text_processed.csv'
silvester_path = '../data/silvester_3_text_processed.csv'
valentine_path = '../data/valentine_3_text_processed.csv'

In [14]:
hockey = pd.read_csv(hockey_path)
silvester = pd.read_csv(silvester_path)
valentine = pd.read_csv(valentine_path)

Fix `datetime` features:

In [15]:
def fix_times(df):
    df['order_time'] = pd.to_datetime(df['order_time'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_time_closed'] = pd.to_datetime(df['order_time_closed'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_item_time'] = pd.to_datetime(df['order_item_time'], format="%Y-%m-%d %H:%M:%S.%f")

In [16]:
fix_times(hockey)
fix_times(valentine)
fix_times(silvester)

Lets only look at interesting features

In [17]:
features = ['order_id', 'order_time', 'order_time_closed',
       'order_item_time', 'title', 'category_id', 
       'dwell_time', 'meal_step',
       'meal_flow_last_to_close', 'total_flow_steps', 'meal_flow_step', 
       'beer_volume', 'order_day_of_week',
       'is_weekend', 'guest_count',
       'period_of_day', 
       'sharable', 'kids_meal', 'birthday',
       'total_sales_before_tax', 'item_qty', 'sales_before_tax']

In [18]:
hockey = hockey[features]
silvester = silvester[features]
valentine = valentine[features]

In [19]:
hockey.head()

,order_id,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
0,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,$Sub.Swt.Pot.Fry-,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,0,0,0,19.84,1.0,2.97
1,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,FINGERS.SAUCED,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,1,0,0,19.84,1.0,16.87
2,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:18:25,TRAD.GUAC,2.0,1.181944,2,13,17,4,0.000,Tuesday,False,1,lunch,1,0,0,26.61,1.0,8.99
3,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:13:26,VEGGIE.BURGER,2.0,1.181944,1,13,17,3,0.000,Tuesday,False,1,lunch,0,0,0,26.61,1.0,17.62
4,520253565,2019-04-16 19:35:00,2019-04-16 22:39:18,2019-04-16 20:56:13,CANADIAN.Pt,1.0,3.071667,2,12,44,19,0.473,Tuesday,False,0,dinner,0,0,0,30.48,1.0,7.62


Create table for each order, we're going to pick (randomly) 10 tables out of each dataframe.

In [20]:
def create_tables(df):
    tables = df.drop_duplicates(subset='order_id', keep='first')
    tables.set_index('order_id', inplace=True)
    tables = tables[['guest_count']]

    return tables

In [21]:
hockey_tables = create_tables(hockey)
valentine_tables = create_tables(valentine)
silvester_tables = create_tables(silvester)

In [22]:
print("We have ", len(hockey_tables), " tables in the Hockey data")

We have  118664  tables in the Hockey data


In [23]:
print("We have ", len(valentine_tables), " tables in the Valentine data")

We have  40787  tables in the Valentine data


In [24]:
print("We have ", len(silvester_tables), " tables in the Silvester data")

We have  92107  tables in the Silvester data


In [25]:
hockey.set_index('order_id', inplace=True)
valentine.set_index('order_id', inplace=True)
silvester.set_index('order_id', inplace=True)

-----

# The Annotations

Pick 10 tables out of each dataframe.

In [26]:
COUNT = 10

In [27]:
import random

In [28]:
def get_tables_gc_6(tables):
    return tables[tables['guest_count'] >5]

## Annotating Hockey

Pick 10 tables of a total of 1 guests of **Hockey**.

In [29]:
hockey_6gc_tables = get_tables_gc_6(hockey_tables)
print("We have {} tables of a total of 6+ guests".format(len(hockey_6gc_tables)))

We have 2377 tables of a total of 6+ guests


In [30]:
picked_hockey_tables = random.sample(list(hockey_6gc_tables.index), COUNT)
picked_hockey_tables

[512719244,
 524400640,
 519632427,
 522735357,
 520780863,
 514598934,
 515905070,
 524213204,
 514317538,
 523327389]

In [79]:
picked_hockey_tables = [514594544,
 520144566,
 512856753,
 523552965,
 522777307,
 520932269,
 520961529,
 522822209,
 524870243,
 514742701,
 521935119,
 514531924,
 513465814
]

# Table 1

In [32]:
k = 0
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

514594544


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Tex Steak Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,15.762830
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,P.B Deli Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,12.805451
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Onion Soup,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,5.904900
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Fish & Chips,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,13.791244
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Fount Pop,4.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,2.947521
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Fish & Chips,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,13.791244
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Tex Steak Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,15.762830
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Hot Beef Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,14.777037
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Chicken Fingers,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,1,0,0,199.603372,1.0,19.686287


# **April 13th (Saturday), 19:58 - 21:23, a total of 1.5 hours**

**Meal Steps Description:**
1. Start
  - Meal very heavy, all ordered at once

2. Coffe 30 min later
  - Dessert

      
**`Dinner`** 


**Explanation**: Because of the time and the size of the meal.

**Features that helped**: the amount of foold on the table is approxemtly the GC, `price of meal` (10 $), and `time of meal` (21:00pm). 

# Table 2

In [33]:
k = 1
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

520144566


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 19:51:43,Deluxe Dinner,2.0,0.773333,1,0,10,0,-0.000,Tuesday,False,22,dinner,0,0,0,3253.84,-1.0,0.000000
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 19:51:43,Deluxe Dinner,2.0,0.773333,1,0,10,0,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,22.0,2259.481435
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 19:51:43,Deluxe Dinner,2.0,0.773333,1,0,10,0,-0.000,Tuesday,False,22,dinner,0,0,0,3253.84,-1.0,0.000000
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Pop,4.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,2.0,5.835438
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Evian,4.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,9.0,91.908142
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Guinness,1.0,0.773333,2,0,10,2,0.440,Tuesday,False,22,dinner,0,0,0,3253.84,1.0,9.336700
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Heineken,1.0,0.773333,2,0,10,2,0.660,Tuesday,False,22,dinner,0,0,0,3253.84,2.0,18.673400
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Castle Cab Sauv,6.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,6.0,406.146456
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,San Pellegrino,4.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,7.0,71.484110


**April 16th (Tuesday), 19:51 - 20:37, total 0.77 hours, 2 meal steps**

** 22 people eat dinner and some order more soft drinks and liquer 
I dont use the the dwell time here
**Meal Steps Description:**
it actually has a single meal flow and not 2 as mentioned. 
       
**Total drinks**: 5 alcholic drinks 3 soft drinks

**Dinner Party**

**Explanation**: This is a party where the foucus is on the food, room was set for the group. 

**Features**: number of dinners which was served for the table, amount of food over the amount of drinks

rule: if the guest count is greather than 12 then it was set before, therefore it a big event and with high ratio of food over drinks it is a dinner event. 

# Table 3

In [34]:
k = 2
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

512856753


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 20:35:00,LG COORS LIGHT,1.0,4.835278,1,8,70,0,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 20:41:00,CHICKEN STIR FRY,2.0,4.835278,2,8,70,1,0.000,Thursday,False,11,dinner,0,0,0,76.75,1.0,16.00
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 21:21:00,LG COORS LIGHT,1.0,4.835278,2,8,70,11,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 22:31:00,LG COORS LIGHT,1.0,4.835278,2,8,70,28,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 23:27:00,LG COORS LIGHT,1.0,4.835278,2,8,70,42,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:28:00,LG COORS LIGHT,1.0,4.835278,2,8,70,57,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:37:00,LG COORS LIGHT,1.0,4.835278,2,8,70,59,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:37:00,LG COORS LIGHT,1.0,4.835278,2,8,70,59,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:52:00,PEPPERONI SML,2.0,4.835278,2,8,70,62,0.000,Thursday,False,11,dinner,0,0,0,76.75,1.0,10.00


**Social Gathering**

**April 11th (Thursday), 20:35 - 01:25, total of 4.83 hours**

**Meal Steps Description:**
1. food and beer
2. 40 minutes later: beer
3. 70 minutes later: beer
4. 60 min later: beer
5. 60 min later: 2 beer
6. 15 min later: pizza


**`Wrong Label`**


**Explanation**: Its hard to say how many people are on the table but it is less than 11 and probabily less than 6 so I classifity this as a wornk label of GC



# Table 4

In [35]:
k = 3
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

523552965


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:42:00,SODA,4.0,1.830556,1,7,26,0,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,3.00
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:42:00,COKE,4.0,1.830556,1,7,26,0,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,3.00
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:50:00,CALAMARI,2.0,1.830556,2,7,26,2,0.00,Saturday,True,6,dinner,1,0,0,188.77,1.0,13.67
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:50:00,CALAMARI,2.0,1.830556,2,7,26,2,0.00,Saturday,True,6,dinner,1,0,0,188.77,1.0,13.67
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,CHICKN FAJITA,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,21.88
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,HALIBUT&CHIPS,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,24.88
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,STEAK FAJITA,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,22.43
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,STEAK FAJITA,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,22.43
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,2LB WINGS,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,1,0,0,188.77,1.0,25.92


**`Dinner`**
Family dinner 


There is also a "kid" meal in here.

many foods and single beer with two soft drinks

**Features**: GC size of a family, rator of food to alcohol beverage is high. small number of soft drinks

# Table 5

In [36]:
k = 4
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

522777307


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,JACK DANIELS DBL,3.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,10.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,CUERVO GOLD,3.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,6.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,CORONA,1.0,0.662778,1,2,8,0,0.355,Saturday,True,6,night,0,0,0,132.45,1.0,7.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,CORONA,1.0,0.662778,1,2,8,0,0.355,Saturday,True,6,night,0,0,0,132.45,1.0,7.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,RED BULL,4.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,4.25
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,HENNESSY DBL,3.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,26.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:24:55,BURT REYNOLDS,3.0,0.662778,2,2,8,4,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,4.95
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:24:55,CORONA,1.0,0.662778,2,2,8,4,0.355,Saturday,True,6,night,0,0,0,132.45,1.0,7.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:24:55,HENNESSY DBL,3.0,0.662778,2,2,8,4,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,26.50


**April 20th (Sauturday), 00:06 - 00:45, total of 0.66 hours**

**`drinking`**

only drinks on the table, 2 meal step and with six rows per meal step. 


**Explanation**: Not much to explain, a group of people are drinking.


# Table 6

In [38]:
k = 5  
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

520932269


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:20:51,POP.,4.0,1.391944,1,8,16,0,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,3.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:20:51,CREEMORE.Pt,1.0,1.391944,1,8,16,0,0.473,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,8.72
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:28:38,POP.,4.0,1.391944,2,8,16,1,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,3.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:28:38,POP.,4.0,1.391944,2,8,16,1,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,3.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:29:39,Loaded.Mash,2.0,1.391944,1,8,16,1,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,0.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:30:17,SAM.ADAM.SEAS.Pt,1.0,1.391944,1,8,16,1,0.597,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,9.33
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:30:17,BELGIAN.MOON..Pt,1.0,1.391944,1,8,16,1,0.473,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,8.43
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:33:39,TUNA.GUAC,2.0,1.391944,1,8,16,1,0.000,Wednesday,False,6,afternoon,1,0,0,111.59,1.0,14.97
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:41:33,FINGERS.SAUCED,2.0,1.391944,2,8,16,2,0.000,Wednesday,False,6,afternoon,1,0,0,111.59,1.0,16.87


**April 17th (Wednesday), 17:20 - 18:43, total of 1.39 hours,.**

**`Dinner`**

**Explanation**: different types of food and varity of drinks, seems like a grown up group gathering after work.
**Features**:  day of week, hour, ration of food to drinks is almost 1



# Table 7

In [39]:
k = 6
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

520961529


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,RICKARD.RED.Pt,1.0,1.639167,1,8,21,0,0.473,Wednesday,False,10,dinner,0,0,0,214.57,1.0,8.43
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,HEINEKEN.Pt,1.0,1.639167,1,8,21,0,0.562,Wednesday,False,10,dinner,0,0,0,214.57,1.0,9.33
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,POP.,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,3.99
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:27,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:27,WATER.,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:27:34,RICKARD.RED.Pt,1.0,1.639167,2,8,21,1,0.473,Wednesday,False,10,dinner,0,0,0,214.57,1.0,8.43


**April 17th (Wednesday), 18:22 - 20:00, total of 1.63 hours**

**`Family`**        

**Explanation**: Its a classic dinner, maybe also watchign a game or something. Starts with a drinks and order food for many people.later order some desserts.


**Features**: staying at the table for 2 hours, many drinks and many food, ratio is almost 0.5  

# Table 8

In [42]:
k = 7
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

522822209


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:48:00,TITO VODKA DBL,3.0,0.503056,1,3,5,0,0.000,Saturday,True,7,night,0,0,0,951.0,12.0,150.00
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:59:00,GOOSE IPA TAP,1.0,0.503056,2,3,5,2,0.591,Saturday,True,7,night,0,0,0,951.0,1.0,7.25
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:59:00,GOOSE IPA TAP,1.0,0.503056,2,3,5,2,0.591,Saturday,True,7,night,0,0,0,951.0,1.0,7.25
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:59:00,GOOSE IPA TAP,1.0,0.503056,2,3,5,2,0.591,Saturday,True,7,night,0,0,0,951.0,1.0,7.25
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:59:00,GOOSE IPA TAP,1.0,0.503056,2,3,5,2,0.591,Saturday,True,7,night,0,0,0,951.0,1.0,7.25
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 02:02:00,1800 SILVER,3.0,0.503056,1,3,5,2,0.000,Saturday,True,7,night,0,0,0,951.0,1.0,7.75
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 02:02:00,1800 SILVER,3.0,0.503056,1,3,5,2,0.000,Saturday,True,7,night,0,0,0,951.0,1.0,7.75
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 02:02:00,1800 SILVER,3.0,0.503056,1,3,5,2,0.000,Saturday,True,7,night,0,0,0,951.0,1.0,7.75
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 02:02:00,TITO VODKA DBL,3.0,0.503056,1,3,5,2,0.000,Saturday,True,7,night,0,0,0,951.0,1.0,12.50


**April 20nd (Saturday), 01:48 - 02:18, total of 0.5 hours**

       
**total drinks**: 4 beers and 10 liquers,

**`Social drinking`**

**Explanation**: Its a drinking table, Its a late night, they came to drink.

**Features**: staying at the table for 30 minutes, food to drink ratio is 0. many drinks, hard liquer

# Table 9

In [41]:
k = 8
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

524870243


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
524870243,2019-04-22 14:52:02,2019-04-22 15:59:35,2019-04-22 14:53:38,Killer Shake,4.0,1.125833,1,11,14,0,0.000,Monday,False,6,afternoon,0,0,0,124.33,1.0,4.99
524870243,2019-04-22 14:52:02,2019-04-22 15:59:35,2019-04-22 14:53:38,Lemonade,4.0,1.125833,1,11,14,0,0.000,Monday,False,6,afternoon,0,0,0,124.33,1.0,2.99
524870243,2019-04-22 14:52:02,2019-04-22 15:59:35,2019-04-22 14:53:38,Bud btl,1.0,1.125833,1,11,14,0,0.341,Monday,False,6,afternoon,0,0,0,124.33,1.0,5.50
524870243,2019-04-22 14:52:02,2019-04-22 15:59:35,2019-04-22 14:53:38,Killer Shake,4.0,1.125833,1,11,14,0,0.000,Monday,False,6,afternoon,0,0,0,124.33,1.0,4.99
524870243,2019-04-22 14:52:02,2019-04-22 15:59:35,2019-04-22 14:53:38,Fount Pop,4.0,1.125833,1,11,14,0,0.000,Monday,False,6,afternoon,0,0,0,124.33,1.0,2.99
524870243,2019-04-22 14:52:02,2019-04-22 15:59:35,2019-04-22 14:53:38,Juice,4.0,1.125833,1,11,14,0,0.000,Monday,False,6,afternoon,0,0,0,124.33,1.0,2.99
524870243,2019-04-22 14:52:02,2019-04-22 15:59:35,2019-04-22 15:00:49,S-Fried Dills,2.0,1.125833,2,11,14,1,0.000,Monday,False,6,afternoon,0,0,0,124.33,1.0,5.99
524870243,2019-04-22 14:52:02,2019-04-22 15:59:35,2019-04-22 15:00:49,S-Mac&Chz Popprs,2.0,1.125833,2,11,14,1,0.000,Monday,False,6,afternoon,0,0,0,124.33,1.0,5.99
524870243,2019-04-22 14:52:02,2019-04-22 15:59:35,2019-04-22 15:03:35,ED ClassicBurger,2.0,1.125833,1,11,14,1,-0.000,Monday,False,6,afternoon,0,0,0,124.33,-1.0,0.00


`Family Event`

# Table 10

In [105]:
k = 9
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

514742701


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
514742701,2019-04-13 18:45:00,2019-04-13 21:57:53,2019-04-13 18:45:00,JUG STELLA,1.0,3.214722,1,28,46,0,1.774,Saturday,True,6,dinner,0,0,0,264.5,1.0,25.5
514742701,2019-04-13 18:45:00,2019-04-13 21:57:53,2019-04-13 18:45:00,JUG STELLA,1.0,3.214722,1,28,46,0,1.774,Saturday,True,6,dinner,0,0,0,264.5,1.0,25.5
514742701,2019-04-13 18:45:00,2019-04-13 21:57:53,2019-04-13 18:47:00,TWELVE WINGS,2.0,3.214722,1,28,46,0,0.000,Saturday,True,6,dinner,1,0,0,264.5,1.0,20.5
514742701,2019-04-13 18:45:00,2019-04-13 21:57:53,2019-04-13 18:47:00,EIGHT WINGS,2.0,3.214722,1,28,46,0,0.000,Saturday,True,6,dinner,1,0,0,264.5,1.0,14.0
514742701,2019-04-13 18:45:00,2019-04-13 21:57:53,2019-04-13 18:47:00,WINGS & FRIES,2.0,3.214722,1,28,46,0,0.000,Saturday,True,6,dinner,1,0,0,264.5,1.0,15.0
514742701,2019-04-13 18:45:00,2019-04-13 21:57:53,2019-04-13 18:47:00,EIGHT WINGS,2.0,3.214722,1,28,46,0,0.000,Saturday,True,6,dinner,1,0,0,264.5,1.0,14.0
514742701,2019-04-13 18:45:00,2019-04-13 21:57:53,2019-04-13 18:47:00,NACHOS SUPREME,2.0,3.214722,1,28,46,0,0.000,Saturday,True,6,dinner,1,0,0,264.5,1.0,8.0
514742701,2019-04-13 18:45:00,2019-04-13 21:57:53,2019-04-13 18:47:00,DELUXE FAJITAS,2.0,3.214722,1,28,46,0,0.000,Saturday,True,6,dinner,0,0,0,264.5,1.0,20.0
514742701,2019-04-13 18:45:00,2019-04-13 21:57:53,2019-04-13 19:26:00,JUG STELLA,1.0,3.214722,2,28,46,9,1.774,Saturday,True,6,dinner,0,0,0,264.5,1.0,25.5


`SOCIAL GATHRING`

# Table 11

In [45]:
k = 10
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

521935119


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521935119,2019-04-19 01:30:00,2019-04-19 02:19:05,2019-04-19 01:30:40,JACK DANIELS,3.0,0.818056,1,0,8,0,0.00,Friday,True,6,night,0,0,0,194.25,1.0,6.25
521935119,2019-04-19 01:30:00,2019-04-19 02:19:05,2019-04-19 01:31:57,HENDRICK LAMAR DBL,3.0,0.818056,1,0,8,0,0.00,Friday,True,6,night,0,0,0,194.25,1.0,15.50
521935119,2019-04-19 01:30:00,2019-04-19 02:19:05,2019-04-19 01:31:57,WELL RYE,3.0,0.818056,1,0,8,0,0.00,Friday,True,6,night,0,0,0,194.25,1.0,6.25
521935119,2019-04-19 01:30:00,2019-04-19 02:19:05,2019-04-19 01:31:57,WELL RYE,3.0,0.818056,1,0,8,0,0.00,Friday,True,6,night,0,0,0,194.25,1.0,6.25
521935119,2019-04-19 01:30:00,2019-04-19 02:19:05,2019-04-19 01:31:57,HENDRICK LAMAR DBL,3.0,0.818056,1,0,8,0,0.00,Friday,True,6,night,0,0,0,194.25,1.0,15.50
521935119,2019-04-19 01:30:00,2019-04-19 02:19:05,2019-04-19 01:31:57,WELL RYE,3.0,0.818056,1,0,8,0,0.00,Friday,True,6,night,0,0,0,194.25,1.0,6.25
521935119,2019-04-19 01:30:00,2019-04-19 02:19:05,2019-04-19 01:31:57,WELL RYE DBL,3.0,0.818056,1,0,8,0,0.00,Friday,True,6,night,0,0,0,194.25,1.0,11.50
521935119,2019-04-19 01:30:00,2019-04-19 02:19:05,2019-04-19 01:47:06,HENDRICK LAMAR DBL,3.0,0.818056,2,0,8,3,0.00,Friday,True,6,night,0,0,0,194.25,1.0,15.50
521935119,2019-04-19 01:30:00,2019-04-19 02:19:05,2019-04-19 01:47:06,HENDRICK LAMAR DBL,3.0,0.818056,2,0,8,3,0.00,Friday,True,6,night,0,0,0,194.25,1.0,15.50


`Drinking`

# Table 12

In [46]:
k = 11
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

514531924


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
514531924,2019-04-13 17:58:47,2019-04-13 19:36:52,2019-04-13 17:58:54,Morning Sunshine,4.0,1.634722,1,0,23,0,0.0,Saturday,True,6,afternoon,0,0,0,175.81,1.0,4.99
514531924,2019-04-13 17:58:47,2019-04-13 19:36:52,2019-04-13 18:15:12,Fish & Chips,2.0,1.634722,2,0,23,4,0.0,Saturday,True,6,afternoon,0,0,0,175.81,1.0,13.99
514531924,2019-04-13 17:58:47,2019-04-13 19:36:52,2019-04-13 18:15:12,Spanish Paella,2.0,1.634722,2,0,23,4,0.0,Saturday,True,6,afternoon,0,0,0,175.81,1.0,17.99
514531924,2019-04-13 17:58:47,2019-04-13 19:36:52,2019-04-13 18:15:12,Salmon Teriyaki,2.0,1.634722,2,0,23,4,0.0,Saturday,True,6,afternoon,0,0,0,175.81,1.0,19.99
514531924,2019-04-13 17:58:47,2019-04-13 19:36:52,2019-04-13 18:15:12,S-Hoisin Bites,2.0,1.634722,2,0,23,4,0.0,Saturday,True,6,afternoon,0,0,0,175.81,1.0,5.99
514531924,2019-04-13 17:58:47,2019-04-13 19:36:52,2019-04-13 18:15:12,Fish & Chips,2.0,1.634722,2,0,23,4,0.0,Saturday,True,6,afternoon,0,0,0,175.81,1.0,13.99
514531924,2019-04-13 17:58:47,2019-04-13 19:36:52,2019-04-13 18:15:12,S-Moroccan Cigrs,2.0,1.634722,2,0,23,4,0.0,Saturday,True,6,afternoon,0,0,0,175.81,1.0,5.99
514531924,2019-04-13 17:58:47,2019-04-13 19:36:52,2019-04-13 18:15:12,Tuna Poke Sal,2.0,1.634722,2,0,23,4,0.0,Saturday,True,6,afternoon,0,0,0,175.81,1.0,17.99
514531924,2019-04-13 17:58:47,2019-04-13 19:36:52,2019-04-13 18:15:12,10oz New York,2.0,1.634722,2,0,23,4,0.0,Saturday,True,6,afternoon,0,0,0,175.81,1.0,25.99


`Dinner`

# Table 13

In [47]:
k = 12
print(picked_hockey_tables[k])
hockey[hockey.index == picked_hockey_tables[k]].sort_values(by=['order_item_time'])

513465814


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
513465814,2019-04-12 17:35:06,2019-04-12 18:45:20,2019-04-12 17:36:16,Meatloaf,2.0,1.170556,1,17,17,0,0.0,Friday,True,7,afternoon,0,0,0,124.89,1.0,13.99
513465814,2019-04-12 17:35:06,2019-04-12 18:45:20,2019-04-12 17:36:16,Mexican Chicken,2.0,1.170556,1,17,17,0,0.0,Friday,True,7,afternoon,0,0,0,124.89,1.0,16.99
513465814,2019-04-12 17:35:06,2019-04-12 18:45:20,2019-04-12 17:36:16,Mexican Chicken,2.0,1.170556,1,17,17,0,0.0,Friday,True,7,afternoon,0,0,0,124.89,1.0,16.99
513465814,2019-04-12 17:35:06,2019-04-12 18:45:20,2019-04-12 17:36:16,Meatloaf,2.0,1.170556,1,17,17,0,0.0,Friday,True,7,afternoon,0,0,0,124.89,1.0,13.99
513465814,2019-04-12 17:35:06,2019-04-12 18:45:20,2019-04-12 17:36:16,Greek LG,2.0,1.170556,1,17,17,0,0.0,Friday,True,7,afternoon,0,0,0,124.89,1.0,17.48
513465814,2019-04-12 17:35:06,2019-04-12 18:45:20,2019-04-12 17:36:16,Spag & Meatballs,2.0,1.170556,1,17,17,0,0.0,Friday,True,7,afternoon,0,0,0,124.89,1.0,15.99
513465814,2019-04-12 17:35:06,2019-04-12 18:45:20,2019-04-12 17:36:16,S-Fried Dills,2.0,1.170556,1,17,17,0,0.0,Friday,True,7,afternoon,0,0,0,124.89,1.0,5.99
513465814,2019-04-12 17:35:06,2019-04-12 18:45:20,2019-04-12 17:36:16,Greek LG,2.0,1.170556,1,17,17,0,0.0,Friday,True,7,afternoon,0,0,0,124.89,1.0,17.48
513465814,2019-04-12 17:35:06,2019-04-12 18:45:20,2019-04-12 17:36:16,S-Mac&Chz Popprs,2.0,1.170556,1,17,17,0,0.0,Friday,True,7,afternoon,0,0,0,124.89,1.0,5.99


`Dinner`

# Valentine

Pick 5 tables of total of 1 guests for **Valentine**.

In [48]:
valentine_gc = get_tables_gc_6(valentine_tables)

print("We have {} tables of a total of 1 guests".format(len(valentine_gc)))

We have 720 tables of a total of 1 guests


In [49]:
import random
picked_valentine_tables = random.sample(list(valentine_gc.index), COUNT)
picked_valentine_tables

[447075207,
 447949648,
 446801321,
 434620563,
 435067240,
 447370109,
 447026354,
 434684899,
 447481789,
 434736097]

In [80]:
picked_valentine_tables = [434693771,
 446750517,
 447075894,
 447589967,
 434539751,
 448045237,
 447558668,
 448730983,
 447983682,
 447100562,
 434734939,
 448721414,
 434681763,
 447350607,
435068108,
435006655,
447350607]
print(len(picked_valentine_tables))


17


## Table 1

In [63]:
k = 0
print(picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

434693771


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 17:38:00,W.KIM.CRAW.SV.1z,6.0,2.141111,1,19,31,0,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,21.00
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 17:51:00,R.ST.JEAN.CAB.Bt,6.0,2.141111,2,19,31,3,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,55.00
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,7OZ.SIRLOIN,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,29.45
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,BACON.FILET*,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,40.95
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,LAMB.SALAD*,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,21.75
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,$Truffle.Fries$$,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,1,0,0,380.5,1.0,3.95
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,CAESAR.APP,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,8.95
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,TUNA.STACK,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,15.50
434693771,2019-02-10 17:38:00,2019-02-10 19:46:28,2019-02-10 18:04:00,b.potato,2.0,2.141111,3,19,31,6,0.0,Sunday,True,7,afternoon,0,0,0,380.5,1.0,0.00


**Feb 10th (Sunday), 17:38 - 19:46, total of ~2 hour, 5 meal steps.**

**total drinks**: 3 wines,

**`family dinner`**


# Table 2

In [64]:
k = 1
print(picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

446750517


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:07:00,LRG BELGIAN MOON,1.0,1.038333,1,6,14,0,0.591,Tuesday,False,8,lunch,0,0,0,57.85,1.0,7.25
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:07:00,HEINEKEN,1.0,1.038333,1,6,14,0,0.330,Tuesday,False,8,lunch,0,0,0,57.85,1.0,7.20
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:18:00,CHEF SPECIAL,2.0,1.038333,2,6,14,2,0.000,Tuesday,False,8,lunch,0,0,0,57.85,1.0,16.97
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:18:00,CHKN CLUBHOUSE,2.0,1.038333,2,6,14,2,0.000,Tuesday,False,8,lunch,0,0,0,57.85,1.0,15.00
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:32:00,HEINEKEN,1.0,1.038333,3,6,14,5,0.330,Tuesday,False,8,lunch,0,0,0,57.85,1.0,7.20
446750517,2019-02-12 12:07:00,2019-02-12 13:09:18,2019-02-12 12:45:00,RG SLEEMAN,1.0,1.038333,4,6,14,8,0.355,Tuesday,False,8,lunch,0,0,0,57.85,1.0,4.23


**Feb 12th (Tuesday), 12:07 - 13:09, total of 1 hour , 4 meal steps.**


       
**total meals**: 2 foods 4 beers

wrong label






# Table 3

In [65]:
k = 2
print(picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

447075894


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,POP.,4.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,3.99
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,BLURASP LEMONADE,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,8.97
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,BAR.MANGER.XL,1.0,1.195,1,12,16,0,0.591,Wednesday,False,9,dinner,0,0,0,232.12,1.0,10.30
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,DBL.BARRL.SHZ.Lg,6.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,17.98
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,BLURASP LEMONADE,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,8.97
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,MARGRITA.LIME,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,8.97
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,SML.RED.SANG,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,13.29
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,STELLA.Pt,1.0,1.195,1,12,16,0,0.591,Wednesday,False,9,dinner,0,0,0,232.12,1.0,9.33
447075894,2019-02-13 18:41:00,2019-02-13 19:52:42,2019-02-13 18:41:00,BEACHED,3.0,1.195,1,12,16,0,0.000,Wednesday,False,9,dinner,0,0,0,232.12,1.0,8.97


**Feb 13th (Wednesday), 13:08 - 14:07, total of ~1 hour, 3 meal steps.**

**total drinks**: 10 
**total food**: 9

**`Dinner`**

**Explanation**: classic Dinner. 

**Features**: time of meal , and spent about an hour.

# Table 4

In [66]:
k = 3
print(picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])


447589967


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:46:00,STELLA,1.0,3.225833,1,46,48,0,0.330,Thursday,False,6,afternoon,0,0,0,84.15,1.0,7.15
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:46:00,LRG RICKARDS,1.0,3.225833,1,46,48,0,0.591,Thursday,False,6,afternoon,0,0,0,84.15,1.0,7.75
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:46:00,LRG BELGIAN MOON,1.0,3.225833,1,46,48,0,0.591,Thursday,False,6,afternoon,0,0,0,84.15,1.0,7.25
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:46:00,1/2 LITRE RED,6.0,3.225833,1,46,48,0,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,18.00
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,STEAK BITES,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,8.00
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,EIGHT WINGS,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,1,0,0,84.15,1.0,14.00
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,1.50,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,1.50
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,CAULIFLOWER BITES,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,6.00
447589967,2019-02-14 17:46:00,2019-02-14 20:59:33,2019-02-14 17:55:00,SAUSAGE MARGESE,2.0,3.225833,2,46,48,2,0.000,Thursday,False,6,afternoon,0,0,0,84.15,1.0,6.50


**`dinner`**

food to drinks ration is higer than 1

time of day

total amout of orders

# Table 5

In [55]:
k = 4
print(picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

434539751


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434539751,2019-02-10 09:38:29,2019-02-10 11:28:21,2019-02-10 09:40:06,K- Bacon&Eggs,2.0,1.831111,1,2,25,0,0.0,Sunday,True,6,breakfast,0,0,0,89.8,1.0,4.49
434539751,2019-02-10 09:38:29,2019-02-10 11:28:21,2019-02-10 09:40:06,Coffee,4.0,1.831111,1,2,25,0,-0.0,Sunday,True,6,breakfast,0,0,0,89.8,-1.0,0.00
434539751,2019-02-10 09:38:29,2019-02-10 11:28:21,2019-02-10 09:40:06,K- Bacon&Eggs,2.0,1.831111,1,2,25,0,0.0,Sunday,True,6,breakfast,0,0,0,89.8,1.0,4.50
434539751,2019-02-10 09:38:29,2019-02-10 11:28:21,2019-02-10 10:49:18,Spin Pepper Oml,2.0,1.831111,2,2,25,17,0.0,Sunday,True,6,breakfast,0,0,0,89.8,1.0,13.99
434539751,2019-02-10 09:38:29,2019-02-10 11:28:21,2019-02-10 10:49:18,Spin Pepper Oml,2.0,1.831111,2,2,25,17,0.0,Sunday,True,6,breakfast,0,0,0,89.8,1.0,16.98
434539751,2019-02-10 09:38:29,2019-02-10 11:28:21,2019-02-10 10:49:18,Muffins,2.0,1.831111,2,2,25,17,0.0,Sunday,True,6,breakfast,0,0,0,89.8,1.0,1.49
434539751,2019-02-10 09:38:29,2019-02-10 11:28:21,2019-02-10 10:49:18,Coffee,4.0,1.831111,2,2,25,17,0.0,Sunday,True,6,breakfast,0,0,0,89.8,1.0,2.69
434539751,2019-02-10 09:38:29,2019-02-10 11:28:21,2019-02-10 10:49:18,Coffee,4.0,1.831111,2,2,25,17,0.0,Sunday,True,6,breakfast,0,0,0,89.8,1.0,2.69
434539751,2019-02-10 09:38:29,2019-02-10 11:28:21,2019-02-10 10:49:18,Pancakes,2.0,1.831111,2,2,25,17,0.0,Sunday,True,6,breakfast,1,0,0,89.8,1.0,10.99


`Family`

# Table 6

In [57]:
k = 5
print(picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

448045237


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
448045237,2019-02-15 18:43:02,2019-02-15 20:24:20,2019-02-15 18:43:09,*Water,4.0,1.688333,1,6,24,0,0.0,Friday,True,6,dinner,0,0,0,119.78,1.0,0.00
448045237,2019-02-15 18:43:02,2019-02-15 20:24:20,2019-02-15 18:44:19,K- Chic Fingers,2.0,1.688333,1,6,24,0,0.0,Friday,True,6,dinner,0,0,0,119.78,1.0,8.99
448045237,2019-02-15 18:43:02,2019-02-15 20:24:20,2019-02-15 18:44:19,K- Burger,2.0,1.688333,1,6,24,0,0.0,Friday,True,6,dinner,0,0,0,119.78,1.0,10.48
448045237,2019-02-15 18:43:02,2019-02-15 20:24:20,2019-02-15 18:55:26,Pesto Chicken,2.0,1.688333,2,6,24,2,0.0,Friday,True,6,dinner,0,0,0,119.78,1.0,18.99
448045237,2019-02-15 18:43:02,2019-02-15 20:24:20,2019-02-15 18:55:26,10oz Peppercorn,2.0,1.688333,2,6,24,2,0.0,Friday,True,6,dinner,0,0,0,119.78,1.0,29.98
448045237,2019-02-15 18:43:02,2019-02-15 20:24:20,2019-02-15 18:55:26,Coffee,4.0,1.688333,2,6,24,2,0.0,Friday,True,6,dinner,0,0,0,119.78,1.0,2.69
448045237,2019-02-15 18:43:02,2019-02-15 20:24:20,2019-02-15 18:55:26,The Fressor,2.0,1.688333,2,6,24,2,0.0,Friday,True,6,dinner,0,0,0,119.78,1.0,16.99
448045237,2019-02-15 18:43:02,2019-02-15 20:24:20,2019-02-15 18:55:26,Deli Meat Plate,2.0,1.688333,2,6,24,2,0.0,Friday,True,6,dinner,0,0,0,119.78,1.0,20.98
448045237,2019-02-15 18:43:02,2019-02-15 20:24:20,2019-02-15 18:55:33,Mojito,3.0,1.688333,2,6,24,2,0.0,Friday,True,6,dinner,0,0,0,119.78,1.0,7.99


`FAmily`

# Table 7

In [68]:
k = 6
print(picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

447558668


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447558668,2019-02-14 21:02:00,2019-02-14 22:23:24,2019-02-14 21:02:00,STELLA.Pt,1.0,1.356667,1,9,19,0,0.591,Thursday,False,6,hang_out,0,0,0,115.32,1.0,9.33
447558668,2019-02-14 21:02:00,2019-02-14 22:23:24,2019-02-14 21:02:00,STELLA.Pt,1.0,1.356667,1,9,19,0,0.591,Thursday,False,6,hang_out,0,0,0,115.32,1.0,9.33
447558668,2019-02-14 21:02:00,2019-02-14 22:23:24,2019-02-14 21:03:00,WINGS.10PC,2.0,1.356667,1,9,19,0,0.000,Thursday,False,6,hang_out,1,0,0,115.32,1.0,16.99
447558668,2019-02-14 21:02:00,2019-02-14 22:23:24,2019-02-14 21:03:00,WINGS,2.0,1.356667,1,9,19,0,0.000,Thursday,False,6,hang_out,1,0,0,115.32,1.0,0.00
447558668,2019-02-14 21:02:00,2019-02-14 22:23:24,2019-02-14 21:06:00,$Gluten.Free.Bun$,2.0,1.356667,1,9,19,0,0.000,Thursday,False,6,hang_out,0,0,0,115.32,1.0,1.00
447558668,2019-02-14 21:02:00,2019-02-14 22:23:24,2019-02-14 21:06:00,HOLY.SMOKE.BURGR,2.0,1.356667,1,9,19,0,0.000,Thursday,False,6,hang_out,0,0,0,115.32,1.0,17.97
447558668,2019-02-14 21:02:00,2019-02-14 22:23:24,2019-02-14 21:06:00,DBL.CHSBURG,2.0,1.356667,1,9,19,0,0.000,Thursday,False,6,hang_out,0,0,0,115.32,1.0,16.68
447558668,2019-02-14 21:02:00,2019-02-14 22:23:24,2019-02-14 21:06:00,QUESADILLAS,2.0,1.356667,1,9,19,0,0.000,Thursday,False,6,hang_out,1,0,0,115.32,1.0,16.55
447558668,2019-02-14 21:02:00,2019-02-14 22:23:24,2019-02-14 21:34:00,STELLA.Pt,1.0,1.356667,2,9,19,7,0.591,Thursday,False,6,hang_out,0,0,0,115.32,1.0,9.33


`DINNER`

# Table 8

In [69]:
k = 7
print(picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

448730983


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
448730983,2019-02-15 23:06:00,2019-02-16 02:08:29,2019-02-15 23:06:00,PB&J,3.0,3.041389,1,8,43,0,0.0,Friday,True,6,night,0,0,0,173.65,1.0,11.75
448730983,2019-02-15 23:06:00,2019-02-16 02:08:29,2019-02-15 23:06:00,PINK LEMONADE DBL,3.0,3.041389,1,8,43,0,0.0,Friday,True,6,night,0,0,0,173.65,1.0,11.50
448730983,2019-02-15 23:06:00,2019-02-16 02:08:29,2019-02-15 23:06:00,WELL VODKA DBL,3.0,3.041389,1,8,43,0,0.0,Friday,True,6,night,0,0,0,173.65,1.0,10.25
448730983,2019-02-15 23:06:00,2019-02-16 02:08:29,2019-02-15 23:32:00,PINK LEMONADE,3.0,3.041389,2,8,43,6,0.0,Friday,True,6,night,0,0,0,173.65,1.0,6.75
448730983,2019-02-15 23:06:00,2019-02-16 02:08:29,2019-02-15 23:32:00,WELL VODKA,3.0,3.041389,2,8,43,6,0.0,Friday,True,6,night,0,0,0,173.65,1.0,5.95
448730983,2019-02-15 23:06:00,2019-02-16 02:08:29,2019-02-15 23:52:00,PB&J,3.0,3.041389,3,8,43,11,0.0,Friday,True,6,night,0,0,0,173.65,1.0,11.75
448730983,2019-02-15 23:06:00,2019-02-16 02:08:29,2019-02-15 23:56:00,WELL VODKA,3.0,3.041389,4,8,43,12,0.0,Friday,True,6,night,0,0,0,173.65,1.0,5.95
448730983,2019-02-15 23:06:00,2019-02-16 02:08:29,2019-02-15 23:56:00,PINK LEMONADE,3.0,3.041389,4,8,43,12,0.0,Friday,True,6,night,0,0,0,173.65,1.0,6.75
448730983,2019-02-15 23:06:00,2019-02-16 02:08:29,2019-02-16 00:01:00,LEMON DROP SHOT,3.0,3.041389,5,8,43,13,0.0,Friday,True,6,night,0,0,0,173.65,1.0,4.95


`Drinking`


# Table 9

In [70]:
print (picked_valentine_tables[8])
valentine[valentine.index == picked_valentine_tables[8]].sort_values(by=['order_item_time'])

447983682


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447983682,2019-02-15 16:37:00,2019-02-15 18:09:00,2019-02-15 16:37:00,LONG.ISLAND,3.0,1.533333,1,16,21,0,0.000,Friday,True,6,afternoon,0,0,0,134.86,1.67,12.37
447983682,2019-02-15 16:37:00,2019-02-15 18:09:00,2019-02-15 16:37:00,BEACHED,3.0,1.533333,1,16,21,0,0.000,Friday,True,6,afternoon,0,0,0,134.86,1.67,12.37
447983682,2019-02-15 16:37:00,2019-02-15 18:09:00,2019-02-15 16:37:00,MARGRITA.LIME,3.0,1.533333,1,16,21,0,0.000,Friday,True,6,afternoon,0,0,0,134.86,1.00,8.97
447983682,2019-02-15 16:37:00,2019-02-15 18:09:00,2019-02-15 16:47:00,QUESADILLAS,2.0,1.533333,2,16,21,2,0.000,Friday,True,6,afternoon,1,0,0,134.86,1.00,14.97
447983682,2019-02-15 16:37:00,2019-02-15 18:09:00,2019-02-15 16:47:00,FRIED.CHK.SAND,2.0,1.533333,2,16,21,2,0.000,Friday,True,6,afternoon,0,0,0,134.86,1.00,17.09
447983682,2019-02-15 16:37:00,2019-02-15 18:09:00,2019-02-15 16:47:00,CHK.CAESAR.BITES,2.0,1.533333,2,16,21,2,0.000,Friday,True,6,afternoon,0,0,0,134.86,1.00,18.96
447983682,2019-02-15 16:37:00,2019-02-15 18:09:00,2019-02-15 16:47:00,Mashed,2.0,1.533333,2,16,21,2,0.000,Friday,True,6,afternoon,0,0,0,134.86,1.00,0.00
447983682,2019-02-15 16:37:00,2019-02-15 18:09:00,2019-02-15 16:47:00,WINGS..,2.0,1.533333,2,16,21,2,0.000,Friday,True,6,afternoon,1,0,0,134.86,1.00,0.00
447983682,2019-02-15 16:37:00,2019-02-15 18:09:00,2019-02-15 16:47:00,WINGS.5PC,2.0,1.533333,2,16,21,2,0.000,Friday,True,6,afternoon,1,0,0,134.86,1.00,10.47


`SOCIAL GATHRING`

# Table 10

In [71]:
k = 9
print (picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

447100562


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447100562,2019-02-13 19:15:00,2019-02-13 20:25:29,2019-02-13 19:15:17,BREAD,2.0,1.174722,1,0,16,0,0.000,Wednesday,False,6,dinner,1,0,0,167.0,1.0,0.00
447100562,2019-02-13 19:15:00,2019-02-13 20:25:29,2019-02-13 19:16:15,NEW YORK STEAK FRITES,2.0,1.174722,1,0,16,0,0.000,Wednesday,False,6,dinner,0,0,0,167.0,1.0,24.50
447100562,2019-02-13 19:15:00,2019-02-13 20:25:29,2019-02-13 19:16:15,KIDS CREAM PASTA,2.0,1.174722,1,0,16,0,0.000,Wednesday,False,6,dinner,0,1,0,167.0,1.0,7.00
447100562,2019-02-13 19:15:00,2019-02-13 20:25:29,2019-02-13 19:16:15,SLATE BURGER,2.0,1.174722,1,0,16,0,0.000,Wednesday,False,6,dinner,0,0,0,167.0,1.0,16.75
447100562,2019-02-13 19:15:00,2019-02-13 20:25:29,2019-02-13 19:16:15,ITALIAN BOLONES SPAGHETTI,2.0,1.174722,1,0,16,0,0.000,Wednesday,False,6,dinner,0,0,0,167.0,1.0,16.50
447100562,2019-02-13 19:15:00,2019-02-13 20:25:29,2019-02-13 19:16:15,SPAGETTI CARBORONARA,2.0,1.174722,1,0,16,0,0.000,Wednesday,False,6,dinner,0,0,0,167.0,1.0,17.75
447100562,2019-02-13 19:15:00,2019-02-13 20:25:29,2019-02-13 19:16:15,ITALIAN BOLONES SPAGHETTI,2.0,1.174722,1,0,16,0,0.000,Wednesday,False,6,dinner,0,0,0,167.0,1.0,16.50
447100562,2019-02-13 19:15:00,2019-02-13 20:25:29,2019-02-13 20:16:18,BUD LIGHT,1.0,1.174722,2,0,16,15,0.341,Wednesday,False,6,dinner,0,0,0,167.0,1.0,6.75
447100562,2019-02-13 19:15:00,2019-02-13 20:25:29,2019-02-13 20:16:18,BUD LIGHT,1.0,1.174722,2,0,16,15,0.341,Wednesday,False,6,dinner,0,0,0,167.0,1.0,6.75


`Dinner`

# Table 11

In [72]:
k = 10
print (picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

434734939


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434734939,2019-02-10 20:27:00,2019-02-10 21:11:57,2019-02-10 20:27:00,KIETHS,1.0,0.749167,1,2,9,0,0.341,Sunday,True,7,dinner,0,0,0,35.0,1.0,5.75
434734939,2019-02-10 20:27:00,2019-02-10 21:11:57,2019-02-10 20:29:00,STEAK BITES,2.0,0.749167,1,2,9,0,0.000,Sunday,True,7,dinner,0,0,0,35.0,1.0,8.00
434734939,2019-02-10 20:27:00,2019-02-10 21:11:57,2019-02-10 20:29:00,GARLICBR/CHEESE,2.0,0.749167,1,2,9,0,0.000,Sunday,True,7,dinner,0,0,0,35.0,1.0,4.00
434734939,2019-02-10 20:27:00,2019-02-10 21:11:57,2019-02-10 20:55:00,KIETHS,1.0,0.749167,2,2,9,6,0.341,Sunday,True,7,dinner,0,0,0,35.0,1.0,5.75
434734939,2019-02-10 20:27:00,2019-02-10 21:11:57,2019-02-10 20:55:00,KIETHS,1.0,0.749167,2,2,9,6,0.341,Sunday,True,7,dinner,0,0,0,35.0,1.0,5.75
434734939,2019-02-10 20:27:00,2019-02-10 21:11:57,2019-02-10 21:01:00,KIETHS,1.0,0.749167,3,2,9,7,0.341,Sunday,True,7,dinner,0,0,0,35.0,1.0,5.75


wrong label

`UNKNOWN`

# Table 12

In [73]:
k = 11
print (picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

448721414


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
448721414,2019-02-15 22:32:00,2019-02-16 02:07:55,2019-02-15 22:32:00,PERONI,1.0,3.598611,1,18,51,0,0.33,Friday,True,6,hang_out,0,0,0,44.9,1.0,6.1
448721414,2019-02-15 22:32:00,2019-02-16 02:07:55,2019-02-15 22:34:00,PERONI,1.0,3.598611,1,18,51,0,0.33,Friday,True,6,hang_out,0,0,0,44.9,1.0,6.1
448721414,2019-02-15 22:32:00,2019-02-16 02:07:55,2019-02-15 23:57:00,PERONI,1.0,3.598611,2,18,51,20,0.33,Friday,True,6,hang_out,0,0,0,44.9,1.0,6.1
448721414,2019-02-15 22:32:00,2019-02-16 02:07:55,2019-02-16 00:03:00,TWELVE WINGS,2.0,3.598611,3,18,51,21,0.00,Friday,True,6,hang_out,1,0,0,44.9,1.0,20.5
448721414,2019-02-15 22:32:00,2019-02-16 02:07:55,2019-02-16 00:53:00,PERONI,1.0,3.598611,4,18,51,33,0.33,Friday,True,6,hang_out,0,0,0,44.9,1.0,6.1


`SOCIAL GATHRING`

# Table 13

In [74]:
k = 12
print (picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

434681763


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434681763,2019-02-10 17:25:01,2019-02-10 18:12:34,2019-02-10 17:25:37,Fettuccine Pomo,2.0,0.7925,1,11,11,0,0.0,Sunday,True,6,afternoon,0,0,0,80.94,1.0,16.99
434681763,2019-02-10 17:25:01,2019-02-10 18:12:34,2019-02-10 17:25:37,Grilled Cheese,2.0,0.7925,1,11,11,0,0.0,Sunday,True,6,afternoon,0,0,0,80.94,1.0,11.99
434681763,2019-02-10 17:25:01,2019-02-10 18:12:34,2019-02-10 17:25:37,K- Salmon,2.0,0.7925,1,11,11,0,0.0,Sunday,True,6,afternoon,0,0,0,80.94,1.0,10.99
434681763,2019-02-10 17:25:01,2019-02-10 18:12:34,2019-02-10 17:25:37,Veggie Burger,2.0,0.7925,1,11,11,0,0.0,Sunday,True,6,afternoon,0,0,0,80.94,1.0,13.99
434681763,2019-02-10 17:25:01,2019-02-10 18:12:34,2019-02-10 17:25:37,K- Ling Pasta,2.0,0.7925,1,11,11,0,0.0,Sunday,True,6,afternoon,0,0,0,80.94,1.0,8.99
434681763,2019-02-10 17:25:01,2019-02-10 18:12:34,2019-02-10 17:25:37,Tuna Poke Bowl,2.0,0.7925,1,11,11,0,0.0,Sunday,True,6,afternoon,0,0,0,80.94,1.0,17.99


`Lunch`

# Table 14 

In [75]:
k = 13
print (picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

447350607


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447350607,2019-02-14 11:27:00,2019-02-14 11:42:24,2019-02-14 11:27:47,ONION RING,2.0,0.256667,1,3,3,0,0.0,Thursday,False,6,lunch,0,0,0,119.5,1.0,6.0
447350607,2019-02-14 11:27:00,2019-02-14 11:42:24,2019-02-14 11:27:47,NY STRIP 12OZ,2.0,0.256667,1,3,3,0,0.0,Thursday,False,6,lunch,0,0,0,119.5,1.0,26.0
447350607,2019-02-14 11:27:00,2019-02-14 11:42:24,2019-02-14 11:27:47,SWEET FRIES,2.0,0.256667,1,3,3,0,0.0,Thursday,False,6,lunch,1,0,0,119.5,1.0,6.0
447350607,2019-02-14 11:27:00,2019-02-14 11:42:24,2019-02-14 11:27:47,FRIES,2.0,0.256667,1,3,3,0,0.0,Thursday,False,6,lunch,1,0,0,119.5,1.0,5.0
447350607,2019-02-14 11:27:00,2019-02-14 11:42:24,2019-02-14 11:27:47,ONION RING,2.0,0.256667,1,3,3,0,0.0,Thursday,False,6,lunch,0,0,0,119.5,1.0,6.0
447350607,2019-02-14 11:27:00,2019-02-14 11:42:24,2019-02-14 11:27:47,CALAMARI,2.0,0.256667,1,3,3,0,0.0,Thursday,False,6,lunch,1,0,0,119.5,1.0,13.0
447350607,2019-02-14 11:27:00,2019-02-14 11:42:24,2019-02-14 11:27:47,PESTO PENNE,2.0,0.256667,1,3,3,0,0.0,Thursday,False,6,lunch,0,0,0,119.5,1.0,14.0
447350607,2019-02-14 11:27:00,2019-02-14 11:42:24,2019-02-14 11:27:47,CALAMARI,2.0,0.256667,1,3,3,0,0.0,Thursday,False,6,lunch,1,0,0,119.5,1.0,13.0
447350607,2019-02-14 11:27:00,2019-02-14 11:42:24,2019-02-14 11:27:47,CHCKN QUESADILLA,2.0,0.256667,1,3,3,0,0.0,Thursday,False,6,lunch,1,0,0,119.5,1.0,13.0


`LUNCH`

# Table 15

In [81]:
k = 14
print (picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

435068108


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
435068108,2019-02-11 18:29:00,2019-02-11 19:13:32,2019-02-11 18:29:00,LG STELLA,1.0,0.742222,1,9,10,0,0.591,Monday,False,9,dinner,0,0,0,52.25,1.0,8.75
435068108,2019-02-11 18:29:00,2019-02-11 19:13:32,2019-02-11 18:29:00,LG CREEMORE,6.0,0.742222,1,9,10,0,0.000,Monday,False,9,dinner,0,0,0,52.25,1.0,7.25
435068108,2019-02-11 18:29:00,2019-02-11 19:13:32,2019-02-11 18:30:00,16 MON WINGS,2.0,0.742222,1,9,10,0,0.000,Monday,False,9,dinner,1,0,0,52.25,1.0,13.00
435068108,2019-02-11 18:29:00,2019-02-11 19:13:32,2019-02-11 18:30:00,16 MON WINGS,2.0,0.742222,1,9,10,0,0.000,Monday,False,9,dinner,1,0,0,52.25,1.0,13.00
435068108,2019-02-11 18:29:00,2019-02-11 19:13:32,2019-02-11 18:35:00,COKE,4.0,0.742222,2,9,10,1,0.000,Monday,False,9,dinner,0,0,0,52.25,1.0,3.25
435068108,2019-02-11 18:29:00,2019-02-11 19:13:32,2019-02-11 18:35:00,8 MON WINGS,2.0,0.742222,2,9,10,1,0.000,Monday,False,9,dinner,1,0,0,52.25,1.0,7.00


`UNKONOW`

# Table 16

In [82]:
k = 15
print (picked_valentine_tables[k])
valentine[valentine.index == picked_valentine_tables[k]].sort_values(by=['order_item_time'])

435006655


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
435006655,2019-02-11 11:42:15,2019-02-11 13:51:17,2019-02-11 11:42:30,Evian,4.0,2.150556,1,3,27,0,0.0,Monday,False,8,lunch,0,0,0,488.25,1.0,8.75
435006655,2019-02-11 11:42:15,2019-02-11 13:51:17,2019-02-11 11:42:30,Bread Basket,2.0,2.150556,1,3,27,0,0.0,Monday,False,8,lunch,1,0,0,488.25,1.0,5.00
435006655,2019-02-11 11:42:15,2019-02-11 13:51:17,2019-02-11 11:42:30,San Pellegrino,4.0,2.150556,1,3,27,0,0.0,Monday,False,8,lunch,0,0,0,488.25,1.0,8.75
435006655,2019-02-11 11:42:15,2019-02-11 13:51:17,2019-02-11 11:42:30,Evian,4.0,2.150556,1,3,27,0,0.0,Monday,False,8,lunch,0,0,0,488.25,1.0,8.75
435006655,2019-02-11 11:42:15,2019-02-11 13:51:17,2019-02-11 11:42:30,Bread Basket,2.0,2.150556,1,3,27,0,0.0,Monday,False,8,lunch,1,0,0,488.25,1.0,5.00
435006655,2019-02-11 11:42:15,2019-02-11 13:51:17,2019-02-11 11:42:30,San Pellegrino,4.0,2.150556,1,3,27,0,0.0,Monday,False,8,lunch,0,0,0,488.25,1.0,8.75
435006655,2019-02-11 11:42:15,2019-02-11 13:51:17,2019-02-11 11:45:58,House Ice Tea,4.0,2.150556,1,3,27,0,0.0,Monday,False,8,lunch,0,0,0,488.25,1.0,4.50
435006655,2019-02-11 11:42:15,2019-02-11 13:51:17,2019-02-11 11:45:58,Pop,4.0,2.150556,1,3,27,0,0.0,Monday,False,8,lunch,0,0,0,488.25,1.0,2.50
435006655,2019-02-11 11:42:15,2019-02-11 13:51:17,2019-02-11 12:27:14,Mushroom Soup,2.0,2.150556,2,3,27,10,0.0,Monday,False,8,lunch,0,0,0,488.25,1.0,12.00


`LUNCH`

# Silvester#
# Silvester#


In [83]:
import random
silvester_gc = get_tables_gc_6(silvester_tables)
print("We have {} tables of a total of 1 guests".format(len(silvester_gc)))

We have 2190 tables of a total of 1 guests


In [89]:
picked_silvester_tables = random.sample(list(silvester_gc.index), 10)


In [90]:
picked_silvester_tables = [361348245, 360970732, 360903046,363236528] + [361588010,
 360052821,
 360026445,
 362956644,
 360848387,
 362889042,
 358979972,
 360542786,
 362306771,
 361348633]
picked_silvester_tables

[361348245,
 360970732,
 360903046,
 363236528,
 361588010,
 360052821,
 360026445,
 362956644,
 360848387,
 362889042,
 358979972,
 360542786,
 362306771,
 361348633]

# Table 1

In [91]:
k = 0
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

361348245


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Fettuccine Pomo,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,16.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,3 Alarm Burger,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,14.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,PB House Burger,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,14.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Great Reuben,5.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,14.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Gnocchi Tomato,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,16.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,S-Taco Baja Fish,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,5.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Mexican Shrimp,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,16.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,Matzo Ball Soup,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,4.99
361348245,2018-12-29 19:05:31,2018-12-29 19:55:18,2018-12-29 19:09:13,P.B Deli Sand,2.0,0.829722,1,11,11,0,0.0,Saturday,True,9,dinner,0,0,0,158.89,1.0,12.99


**DEC 29th (Saturday), 19:05 - 19:55, total of 50 minutes, 1 meal steps.**

**`dinner`** 

`dinner_time`, `dewll_meal`


# Table 2

In [92]:
k = 1
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

360970732


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:40:10,Green Monster,4.0,1.356944,1,6,18,0,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,5.49
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:40:10,Killer Shake,4.0,1.356944,1,6,18,0,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,4.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,Maple Nut Salmon,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,18.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,Fettuccine Pomo,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,18.48
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,Spag & Meatballs,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,15.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,K- Spag&Meatball,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,9.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,S-Mac&Chz Popprs,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,5.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 21:49:23,Maple Nut Salmon,2.0,1.356944,2,6,18,2,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,18.99
360970732,2018-12-28 21:39:51,2018-12-28 23:01:16,2018-12-28 22:03:36,S-Moroccan Cigrs,2.0,1.356944,3,6,18,5,0.0,Friday,True,6,hang_out,0,0,0,130.87,1.0,5.99


**Dec 28th (Friday), 21:39 - 23:01, 1.35 hrs, 4 meal steps.**

**Total drinks**: No Drinks

**`Family Occasion`** 

late night, no drinks, K dish



# Table 3

In [93]:
k = 2
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

360903046


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,CRAB.CAKES*,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,1,0,0,249.2,1.0,17.95
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,PF-CHEESECAKE,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,1,0,0,249.2,1.0,0.00
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,PF-CHEESECAKE,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,1,0,0,249.2,1.0,0.00
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,CHICKEN.CAESAR,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,17.95
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,R.BER.CAL.CAB.1z,6.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,14.50
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,GRILL.RIBS,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,31.95
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,PF-GARDEN.SALAD,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,0.00
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,PF-PAN.SALMON.,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,40.00
360903046,2018-12-28 19:25:00,2018-12-28 20:46:29,2018-12-28 19:30:00,CHKN.COBB.SALAD,2.0,1.358056,1,19,20,1,0.0,Friday,True,7,dinner,0,0,0,249.2,1.0,14.95


**Dec 28th (Friday), 19:25 - 20:46, 1.35 hrs, 1 meal steps.**

**Total drinks**: 1 wine

**`dinner`** 

`late night`, `1 drink` 




# Table 4

In [94]:
k = 3
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

363236528


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
363236528,2019-01-03 13:16:16,2019-01-03 14:07:17,2019-01-03 13:17:08,The B.B.B.L.T.,2.0,0.850278,1,1,12,0,0.0,Thursday,False,8,lunch,0,0,0,118.09,1.0,14.803220
363236528,2019-01-03 13:16:16,2019-01-03 14:07:17,2019-01-03 13:17:08,Veggie Burger,2.0,0.850278,1,1,12,0,0.0,Thursday,False,8,lunch,0,0,0,118.09,1.0,13.815681
363236528,2019-01-03 13:16:16,2019-01-03 14:07:17,2019-01-03 13:17:08,Classic Burger,2.0,0.850278,1,1,12,0,0.0,Thursday,False,8,lunch,0,0,0,118.09,1.0,15.771009
363236528,2019-01-03 13:16:16,2019-01-03 14:07:17,2019-01-03 13:17:08,Meatloaf,2.0,0.850278,1,1,12,0,0.0,Thursday,False,8,lunch,0,0,0,118.09,1.0,13.815681
363236528,2019-01-03 13:16:16,2019-01-03 14:07:17,2019-01-03 13:17:08,3 Alarm Burger,2.0,0.850278,1,1,12,0,0.0,Thursday,False,8,lunch,0,0,0,118.09,1.0,14.803220
363236528,2019-01-03 13:16:16,2019-01-03 14:07:17,2019-01-03 13:17:08,Super Berry Kale,2.0,0.850278,1,1,12,0,0.0,Thursday,False,8,lunch,0,0,0,118.09,1.0,13.815681
363236528,2019-01-03 13:16:16,2019-01-03 14:07:17,2019-01-03 13:17:08,JOJO's Shrimp,2.0,0.850278,1,1,12,0,0.0,Thursday,False,8,lunch,0,0,0,118.09,1.0,15.790760
363236528,2019-01-03 13:16:16,2019-01-03 14:07:17,2019-01-03 13:17:08,K- Spag&Meatball,2.0,0.850278,1,1,12,0,0.0,Thursday,False,8,lunch,0,0,0,118.09,1.0,9.865522
363236528,2019-01-03 13:16:16,2019-01-03 14:07:17,2019-01-03 13:17:08,Coffee,4.0,0.850278,1,1,12,0,0.0,Thursday,False,8,lunch,0,0,0,118.09,1.0,2.656482


**Jan 3rd (Thursday), 13:16 - 14:07, 0.85 hrs, 2 meal steps.**

**Total drinks**: 

**`Family`** 



# Table 5

In [96]:
k = 4
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

361588010


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
361588010,2018-12-30 01:04:52,2018-12-30 02:38:40,2018-12-30 01:04:52,PINT ORGANIC,1.0,1.563333,1,23,23,0,0.591,Sunday,True,6,night,0,0,0,14.82,1.0,7.08
361588010,2018-12-30 01:04:52,2018-12-30 02:38:40,2018-12-30 01:04:52,STELLA PINT,1.0,1.563333,1,23,23,0,0.591,Sunday,True,6,night,0,0,0,14.82,1.0,7.74


`UNKNOWN`

# Table 6

In [97]:
k = 5
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

360052821


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360052821,2018-12-26 20:02:41,2018-12-26 20:55:06,2018-12-26 20:02:49,16oz Sapporo SPC,1.0,0.873611,1,9,12,0,0.473,Wednesday,False,6,dinner,0,0,0,104.39,1.0,5.99
360052821,2018-12-26 20:02:41,2018-12-26 20:55:06,2018-12-26 20:02:49,Pina Colada SPC,3.0,0.873611,1,9,12,0,0.000,Wednesday,False,6,dinner,0,0,0,104.39,1.0,6.50
360052821,2018-12-26 20:02:41,2018-12-26 20:55:06,2018-12-26 20:16:13,K- Shake Choc,4.0,0.873611,2,9,12,3,0.000,Wednesday,False,6,dinner,0,0,0,104.39,1.0,2.99
360052821,2018-12-26 20:02:41,2018-12-26 20:55:06,2018-12-26 20:16:13,K- Shake Choc,4.0,0.873611,2,9,12,3,0.000,Wednesday,False,6,dinner,0,0,0,104.39,1.0,2.99
360052821,2018-12-26 20:02:41,2018-12-26 20:55:06,2018-12-26 20:16:13,K- Shake Van,4.0,0.873611,2,9,12,3,0.000,Wednesday,False,6,dinner,0,0,0,104.39,1.0,2.99
360052821,2018-12-26 20:02:41,2018-12-26 20:55:06,2018-12-26 20:17:33,Steak & Eggs,2.0,0.873611,2,9,12,3,0.000,Wednesday,False,6,dinner,0,0,0,104.39,1.0,15.99
360052821,2018-12-26 20:02:41,2018-12-26 20:55:06,2018-12-26 20:17:33,Steak & Eggs,2.0,0.873611,2,9,12,3,0.000,Wednesday,False,6,dinner,0,0,0,104.39,1.0,18.98
360052821,2018-12-26 20:02:41,2018-12-26 20:55:06,2018-12-26 20:17:33,K- Penne Pasta,2.0,0.873611,2,9,12,3,0.000,Wednesday,False,6,dinner,0,0,0,104.39,1.0,8.99
360052821,2018-12-26 20:02:41,2018-12-26 20:55:06,2018-12-26 20:17:33,K- Chic Fingers,2.0,0.873611,2,9,12,3,0.000,Wednesday,False,6,dinner,0,0,0,104.39,1.0,9.99


`FAMILY`

# Table 7

In [98]:
k = 6
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

360026445


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360026445,2018-12-26 17:38:00,2018-12-26 18:06:21,2018-12-26 17:38:00,BAJA FISH TACOS,2.0,0.4725,1,7,7,0,0.0,Wednesday,False,6,afternoon,0,0,0,35.7,1.0,5.95
360026445,2018-12-26 17:38:00,2018-12-26 18:06:21,2018-12-26 17:38:00,MAC 'N CHEESE,2.0,0.4725,1,7,7,0,0.0,Wednesday,False,6,afternoon,0,0,0,35.7,1.0,5.95
360026445,2018-12-26 17:38:00,2018-12-26 18:06:21,2018-12-26 17:38:00,BAJA FISH TACOS,2.0,0.4725,1,7,7,0,0.0,Wednesday,False,6,afternoon,0,0,0,35.7,1.0,5.95
360026445,2018-12-26 17:38:00,2018-12-26 18:06:21,2018-12-26 17:38:00,WINGS - S AND P,2.0,0.4725,1,7,7,0,0.0,Wednesday,False,6,afternoon,1,0,0,35.7,1.0,5.95
360026445,2018-12-26 17:38:00,2018-12-26 18:06:21,2018-12-26 17:38:00,PEROGIES,2.0,0.4725,1,7,7,0,0.0,Wednesday,False,6,afternoon,0,0,0,35.7,1.0,5.95
360026445,2018-12-26 17:38:00,2018-12-26 18:06:21,2018-12-26 17:38:00,MAC 'N CHEESE,2.0,0.4725,1,7,7,0,0.0,Wednesday,False,6,afternoon,0,0,0,35.7,1.0,5.95


# Table 8

In [99]:
k = 7
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

362956644


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
362956644,2019-01-02 18:58:28,2019-01-02 20:07:52,2019-01-02 18:58:34,Asian Nachos,2.0,1.156667,1,15,16,0,0.0,Wednesday,False,7,dinner,1,0,0,146.88,0.25,3.25
362956644,2019-01-02 18:58:28,2019-01-02 20:07:52,2019-01-02 18:58:34,Asian Nachos,2.0,1.156667,1,15,16,0,0.0,Wednesday,False,7,dinner,1,0,0,146.88,0.25,3.25
362956644,2019-01-02 18:58:28,2019-01-02 20:07:52,2019-01-02 18:58:34,Asian Nachos,2.0,1.156667,1,15,16,0,0.0,Wednesday,False,7,dinner,1,0,0,146.88,0.25,3.25
362956644,2019-01-02 18:58:28,2019-01-02 20:07:52,2019-01-02 18:58:34,Asian Nachos,2.0,1.156667,1,15,16,0,0.0,Wednesday,False,7,dinner,1,0,0,146.88,0.25,3.24
362956644,2019-01-02 18:58:28,2019-01-02 20:07:52,2019-01-02 19:04:44,Fount Pop,4.0,1.156667,2,15,16,1,0.0,Wednesday,False,7,dinner,0,0,0,146.88,1.00,2.99
362956644,2019-01-02 18:58:28,2019-01-02 20:07:52,2019-01-02 19:04:44,Fettuccine Pomo,2.0,1.156667,2,15,16,1,0.0,Wednesday,False,7,dinner,0,0,0,146.88,1.00,16.99
362956644,2019-01-02 18:58:28,2019-01-02 20:07:52,2019-01-02 19:04:44,Veggie Burger,2.0,1.156667,2,15,16,1,0.0,Wednesday,False,7,dinner,0,0,0,146.88,1.00,13.99
362956644,2019-01-02 18:58:28,2019-01-02 20:07:52,2019-01-02 19:04:44,Fount Pop,4.0,1.156667,2,15,16,1,0.0,Wednesday,False,7,dinner,0,0,0,146.88,1.00,2.99
362956644,2019-01-02 18:58:28,2019-01-02 20:07:52,2019-01-02 19:04:44,10oz New York,2.0,1.156667,2,15,16,1,0.0,Wednesday,False,7,dinner,0,0,0,146.88,1.00,25.99


`DINNER`

# Table 9

In [100]:
k = 8
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

360848387


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360848387,2018-12-28 16:58:00,2018-12-28 18:34:10,2018-12-28 16:58:00,V.CAESAR,4.0,1.602778,1,10,22,0,0.0,Friday,True,6,afternoon,0,0,0,161.56,1.0,4.95
360848387,2018-12-28 16:58:00,2018-12-28 18:34:10,2018-12-28 16:58:00,V.STRAWBERRY.DAQ,4.0,1.602778,1,10,22,0,0.0,Friday,True,6,afternoon,0,0,0,161.56,1.0,4.95
360848387,2018-12-28 16:58:00,2018-12-28 18:34:10,2018-12-28 16:58:00,CHOC.SHAKE,4.0,1.602778,1,10,22,0,0.0,Friday,True,6,afternoon,0,0,0,161.56,1.0,5.92
360848387,2018-12-28 16:58:00,2018-12-28 18:34:10,2018-12-28 16:58:00,DAQUIRI,3.0,1.602778,1,10,22,0,0.0,Friday,True,6,afternoon,0,0,0,161.56,1.0,8.56
360848387,2018-12-28 16:58:00,2018-12-28 18:34:10,2018-12-28 17:01:00,POLAR.ICE*,3.0,1.602778,1,10,22,0,0.0,Friday,True,6,afternoon,0,0,0,161.56,2.0,11.46
360848387,2018-12-28 16:58:00,2018-12-28 18:34:10,2018-12-28 17:21:00,FINGERS.SAUCED,2.0,1.602778,2,10,22,5,0.0,Friday,True,6,afternoon,1,0,0,161.56,1.0,16.87
360848387,2018-12-28 16:58:00,2018-12-28 18:34:10,2018-12-28 17:21:00,DRUNKEN.PLATTER.,2.0,1.602778,2,10,22,5,0.0,Friday,True,6,afternoon,0,0,0,161.56,1.0,0.00
360848387,2018-12-28 16:58:00,2018-12-28 18:34:10,2018-12-28 17:21:00,HOLY.SMOKE.BURGR,2.0,1.602778,2,10,22,5,0.0,Friday,True,6,afternoon,0,0,0,161.56,1.0,17.97
360848387,2018-12-28 16:58:00,2018-12-28 18:34:10,2018-12-28 17:21:00,DBL.BAC.CHSBRG,2.0,1.602778,2,10,22,5,0.0,Friday,True,6,afternoon,0,0,0,161.56,1.0,17.87


`DINNER`

# Table 10

In [101]:
k = 9
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

362889042


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
362889042,2019-01-02 13:38:12,2019-01-02 14:52:26,2019-01-02 13:40:30,PB Nachos,2.0,1.237222,1,2,15,0,0.0,Wednesday,False,6,lunch,1,0,0,136.84,1.0,14.99
362889042,2019-01-02 13:38:12,2019-01-02 14:52:26,2019-01-02 13:40:30,Classic Burger,2.0,1.237222,1,2,15,0,0.0,Wednesday,False,6,lunch,0,0,0,136.84,1.0,14.48
362889042,2019-01-02 13:38:12,2019-01-02 14:52:26,2019-01-02 13:40:30,Classic Burger,2.0,1.237222,1,2,15,0,0.0,Wednesday,False,6,lunch,0,0,0,136.84,1.0,14.48
362889042,2019-01-02 13:38:12,2019-01-02 14:52:26,2019-01-02 13:40:30,Super Berry Kale,2.0,1.237222,1,2,15,0,0.0,Wednesday,False,6,lunch,0,0,0,136.84,1.0,18.98
362889042,2019-01-02 13:38:12,2019-01-02 14:52:26,2019-01-02 13:40:30,Classic Burger,2.0,1.237222,1,2,15,0,0.0,Wednesday,False,6,lunch,0,0,0,136.84,1.0,12.99
362889042,2019-01-02 13:38:12,2019-01-02 14:52:26,2019-01-02 13:40:30,Pesto Chicken,2.0,1.237222,1,2,15,0,0.0,Wednesday,False,6,lunch,0,0,0,136.84,1.0,18.99
362889042,2019-01-02 13:38:12,2019-01-02 14:52:26,2019-01-02 13:40:30,Classic Burger,2.0,1.237222,1,2,15,0,-0.0,Wednesday,False,6,lunch,0,0,0,136.84,-1.0,0.00
362889042,2019-01-02 13:38:12,2019-01-02 14:52:26,2019-01-02 13:40:30,Spanish Paella,2.0,1.237222,1,2,15,0,0.0,Wednesday,False,6,lunch,0,0,0,136.84,1.0,17.99
362889042,2019-01-02 13:38:12,2019-01-02 14:52:26,2019-01-02 14:22:13,Fount Pop,4.0,1.237222,2,2,15,10,0.0,Wednesday,False,6,lunch,0,0,0,136.84,1.0,2.99


`LUNCH`

# Table 11

In [102]:
k = 10
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

358979972


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
358979972,2018-12-23 18:11:00,2018-12-23 19:11:30,2018-12-23 18:11:00,POP.,4.0,1.008333,1,11,14,0,0.000,Sunday,True,6,dinner,0,0,0,155.04,1.0,3.99
358979972,2018-12-23 18:11:00,2018-12-23 19:11:30,2018-12-23 18:11:00,POP.,4.0,1.008333,1,11,14,0,0.000,Sunday,True,6,dinner,0,0,0,155.04,1.0,3.99
358979972,2018-12-23 18:11:00,2018-12-23 19:11:30,2018-12-23 18:11:00,BRUSCHETTA.,2.0,1.008333,1,11,14,0,0.000,Sunday,True,6,dinner,0,0,0,155.04,1.0,13.31
358979972,2018-12-23 18:11:00,2018-12-23 19:11:30,2018-12-23 18:11:00,COORS.LIGHT.Pt,1.0,1.008333,1,11,14,0,0.473,Sunday,True,6,dinner,0,0,0,155.04,1.0,7.62
358979972,2018-12-23 18:11:00,2018-12-23 19:11:30,2018-12-23 18:11:00,JACK'S.PAN&CHEES,2.0,1.008333,1,11,14,0,0.000,Sunday,True,6,dinner,0,0,0,155.04,1.0,8.99
358979972,2018-12-23 18:11:00,2018-12-23 19:11:30,2018-12-23 18:11:00,POP.,4.0,1.008333,1,11,14,0,0.000,Sunday,True,6,dinner,0,0,0,155.04,1.0,3.99
358979972,2018-12-23 18:11:00,2018-12-23 19:11:30,2018-12-23 18:11:00,WATER.,4.0,1.008333,1,11,14,0,0.000,Sunday,True,6,dinner,0,0,0,155.04,1.0,0.00
358979972,2018-12-23 18:11:00,2018-12-23 19:11:30,2018-12-23 18:11:00,WATER.,4.0,1.008333,1,11,14,0,0.000,Sunday,True,6,dinner,0,0,0,155.04,1.0,0.00
358979972,2018-12-23 18:11:00,2018-12-23 19:11:30,2018-12-23 18:26:00,CHICKEN.CLUB,2.0,1.008333,2,11,14,3,0.000,Sunday,True,6,dinner,0,0,0,155.04,1.0,16.98


`DINNER`

# Table 12

In [103]:
k = 11
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

360542786


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360542786,2018-12-27 17:19:00,2018-12-27 20:31:20,2018-12-27 17:19:00,POP.,4.0,3.205556,1,19,45,0,0.000,Thursday,False,6,afternoon,0,0,0,229.02,1.0,3.99
360542786,2018-12-27 17:19:00,2018-12-27 20:31:20,2018-12-27 17:20:00,COORS.LIGHT.16,1.0,3.205556,1,19,45,0,0.444,Thursday,False,6,afternoon,0,0,0,229.02,1.0,7.25
360542786,2018-12-27 17:19:00,2018-12-27 20:31:20,2018-12-27 17:20:00,POP.,4.0,3.205556,1,19,45,0,0.000,Thursday,False,6,afternoon,0,0,0,229.02,1.0,3.99
360542786,2018-12-27 17:19:00,2018-12-27 20:31:20,2018-12-27 17:20:00,POP.,4.0,3.205556,1,19,45,0,0.000,Thursday,False,6,afternoon,0,0,0,229.02,1.0,3.99
360542786,2018-12-27 17:19:00,2018-12-27 20:31:20,2018-12-27 17:23:00,W.MANIAC.RIES.Gl,6.0,3.205556,1,19,45,0,0.000,Thursday,False,6,afternoon,0,0,0,229.02,1.0,9.20
360542786,2018-12-27 17:19:00,2018-12-27 20:31:20,2018-12-27 17:23:00,W.MANIAC.RIES.Gl,6.0,3.205556,1,19,45,0,0.000,Thursday,False,6,afternoon,0,0,0,229.02,1.0,9.20
360542786,2018-12-27 17:19:00,2018-12-27 20:31:20,2018-12-27 17:23:00,W.J.T.CHARD.Gl,6.0,3.205556,1,19,45,0,0.000,Thursday,False,6,afternoon,0,0,0,229.02,1.0,7.75
360542786,2018-12-27 17:19:00,2018-12-27 20:31:20,2018-12-27 17:33:00,CHEESE.BURGER,2.0,3.205556,2,19,45,2,0.000,Thursday,False,6,afternoon,0,0,0,229.02,1.0,17.95
360542786,2018-12-27 17:19:00,2018-12-27 20:31:20,2018-12-27 17:33:00,PRIME.RIB.SNDWIC,2.0,3.205556,2,19,45,2,0.000,Thursday,False,6,afternoon,0,0,0,229.02,1.0,21.95


`DINNER`

# Table 13

In [104]:
k = 12
print (picked_silvester_tables[k])
silvester[silvester.index == picked_silvester_tables[k]].sort_values(by=['order_item_time'])

362306771


,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
362306771,2018-12-31 20:41:00,2018-12-31 22:00:28,2018-12-31 20:41:00,CORONA MONDAY,1.0,1.324444,1,13,18,0,0.355,Monday,False,6,dinner,0,0,0,83.8,1.0,5.00
362306771,2018-12-31 20:41:00,2018-12-31 22:00:28,2018-12-31 20:41:00,CORONA MONDAY,1.0,1.324444,1,13,18,0,0.355,Monday,False,6,dinner,0,0,0,83.8,1.0,5.00
362306771,2018-12-31 20:41:00,2018-12-31 22:00:28,2018-12-31 20:41:00,CORONA MONDAY,1.0,1.324444,1,13,18,0,0.355,Monday,False,6,dinner,0,0,0,83.8,1.0,5.00
362306771,2018-12-31 20:41:00,2018-12-31 22:00:28,2018-12-31 20:41:00,CORONA MONDAY,1.0,1.324444,1,13,18,0,0.355,Monday,False,6,dinner,0,0,0,83.8,1.0,5.00
362306771,2018-12-31 20:41:00,2018-12-31 22:00:28,2018-12-31 20:41:00,CORONA MONDAY,1.0,1.324444,1,13,18,0,0.355,Monday,False,6,dinner,0,0,0,83.8,1.0,5.00
362306771,2018-12-31 20:41:00,2018-12-31 22:00:28,2018-12-31 20:41:00,CORONA MONDAY,1.0,1.324444,1,13,18,0,0.355,Monday,False,6,dinner,0,0,0,83.8,1.0,5.00
362306771,2018-12-31 20:41:00,2018-12-31 22:00:28,2018-12-31 20:50:00,SOFT TACO (CHICK,2.0,1.324444,2,13,18,2,0.000,Monday,False,6,dinner,0,0,0,83.8,1.0,5.95
362306771,2018-12-31 20:41:00,2018-12-31 22:00:28,2018-12-31 20:50:00,WORKS BURGER,2.0,1.324444,2,13,18,2,0.000,Monday,False,6,dinner,0,0,0,83.8,1.0,5.95
362306771,2018-12-31 20:41:00,2018-12-31 22:00:28,2018-12-31 20:50:00,MUSHROOM BURGER,2.0,1.324444,2,13,18,2,0.000,Monday,False,6,dinner,0,0,0,83.8,1.0,5.95


`SOCIAL GATHRING`